In [1]:
%load_ext autoreload
%autoreload 2

In [232]:
from matplotlib import pyplot as plt
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [327]:
from dataset import Dataset
from models import UNet
from Trainer import Trainer
from utils import accuracy
from utils import save_predictions_as_imgs
from utils import hard_dice
from utils import DiceLoss
from utils import make_blending
from utils import BCEDiceLoss

In [441]:
train_folder = 'train'
batch_size = 1
device = 'cuda'

train_transforms = A.Compose(
    [
        A.Resize(height=512, width=512),
        #A.Rotate(limit=10, p=1),
        #A.HorizontalFlip(p=0.5),
        #A.VerticalFlip(p=0.5),
        #A.Blur(blur_limit=11, p=1),
        #A.ChannelShuffle(),
        #A.MedianBlur(blur_limit=1, p=1.0),
        #A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0, p=1),
        #A.ColorJitter(),
        # Это реальные среднее и дисперсия выборки, но такая нормализация не сильно помогает
        #tensor([0.8418, 0.8288, 0.8200]), tensor([0.2174, 0.2178, 0.2234]))
        #A.Normalize(mean=([0.8418, 0.8288, 0.8200]), std=([0.2174, 0.2178, 0.2234])),
        #A.Normalize(),
        ToTensorV2()
    ])

train_dataset = Dataset(train_folder, train_transforms)
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size, pin_memory=True, shuffle=True)

In [498]:
model = UNet(n_filters=32)
#criterion = nn.BCEWithLogitsLoss()
criterion = DiceLoss()
metric = {'name' : 'dice', 'func' : hard_dice}
config = {
    'lr': 1e-3,
    'epochs': 40,
    'early_stopping': 20
}
trainer = Trainer(model, criterion, metric, config)

In [ ]:
trainer.fit(train_loader)

Epoch 1:   0%|                                                                                  | 0/22 [00:00<?, ?it/s]

Epoch 0, loss: 0.3947928168556907,                   dice: 0.6764203228733756


Epoch 2:   0%|                                                                                  | 0/22 [00:00<?, ?it/s]

Epoch 1, loss: 0.29843450405380945,                   dice: 0.7785560285503214


Epoch 3:   0%|                                                                                  | 0/22 [00:00<?, ?it/s]

...

Epoch 25:   0%|                                                                                 | 0/22 [00:00<?, ?it/s]

Epoch 24, loss: 0.14430101622234692,                   dice: 0.8696328862146898


Epoch 26:   0%|                                                                                 | 0/22 [00:00<?, ?it/s]

Epoch 25, loss: 0.1500673917206851,                   dice: 0.8565067459236492


Epoch 26:  91%|█████████████████████████████████████████████████████████████████▍      | 20/22 [00:10<00:01,  1.68it/s]

In [ ]:
save_predictions_as_imgs(train_loader, model)

In [496]:
model.load_state_dict(torch.load(os.path.join(Trainer.CHECKPOINTS_PATH, 'weights.pth')))
save_predictions_as_imgs(train_loader, model)

In [492]:
def show_images_with_mask(number, model_mask=False):
    img_path = './saved_images/orig_' + str(number) + '.png'
    mask_path = ''
    if model_mask == False:
        mask_path = './saved_images/' + str(number) + '.png'
    else:
        mask_path = './saved_images/pred_' + str(number) + '.png'
    plt.figure(figsize=(7, 7))
    blend = make_blending(img_path, mask_path)
    plt.axis('off')
    plt.imshow(blend)

In [493]:
show_images_with_mask(6, False)
show_images_with_mask(6, True)

FileNotFoundError: No such file: 'C:\Users\gefre\Desktop\Sezino\ШИФТ\SegmentationProject\saved_images\orig_6.png'

<Figure size 504x504 with 0 Axes>